In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import pickle

import numpy as np
import pandas as pd

import settings as conf

In [3]:
output_dir = os.path.join(conf.DELIVERABLES_DIR, 'roc_validation', 'classifier_tables')
os.makedirs(output_dir, exist_ok=True)

# Load gene mappings

In [4]:
with open(os.path.join(conf.GENES_METADATA_DIR, 'genes_mapping_simplified-0.pkl'), 'rb') as f:
    genes_mapping_0 = pickle.load(f)

with open(os.path.join(conf.GENES_METADATA_DIR, 'genes_mapping_simplified-1.pkl'), 'rb') as f:
    genes_mapping_1 = pickle.load(f)

# Load fastENLOC results

In [5]:
spredixcan_genes_associations_filename = os.path.join(conf.GENE_ASSOC_DIR, 'fastenloc-torus-rcp.pkl.xz')
display(spredixcan_genes_associations_filename)

spredixcan_genes_associations = pd.read_pickle(spredixcan_genes_associations_filename)

'/mnt/phenomexcan_base/gene_assoc/fastenloc-torus-rcp.pkl.xz'

In [6]:
#spredixcan_genes_associations = spredixcan_genes_associations.rename(index=genes_mapping_1)

In [7]:
spredixcan_genes_associations.shape

(38062, 4091)

In [8]:
spredixcan_genes_associations.head(5)

,O46-Diagnoses_main_ICD10_O46_Antepartum_haemorrhage_not_elsewhere_classified,K30-Diagnoses_main_ICD10_K30_Dyspepsia,2907-Ever_stopped_smoking_for_6_months,H7_DIPLOPIA-Diplopia,1538_0-Major_dietary_changes_in_the_last_5_years_No,5663-Length_of_longest_manicirritable_episode,20002_1538-Noncancer_illness_code_selfreported_arthritis_nos,S30-Diagnoses_main_ICD10_S30_Superficial_injury_of_abdomen_lower_back_and_pelvis,24010_raw-Inverse_distance_to_the_nearest_road,3143_raw-Ankle_spacing_width,...,2237-Plays_computer_games,20002_1461-Noncancer_illness_code_selfreported_inflammatory_bowel_disease,20002_1508-Noncancer_illness_code_selfreported_jaundice_unknown_cause,20003_1140881882-Treatmentmedication_code_timoptol_025_eye_drops,22601_71253330-Job_coding_merchandiser_window_dresser,23112_raw-Leg_fat_mass_right,20003_1140861778-Treatmentmedication_code_dipyridamole,20003_1199-Treatmentmedication_code_food_supplementplantherbal_extract,1309-Fresh_fruit_intake,100920_2105-Type_milk_consumed_soya_with_calcium
gene_id,,,,,,,,,,,,,,,,,,,,,
ENSG00000000419,NaN,NaN,NaN,NaN,0.001213,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.002131,NaN
ENSG00000000457,NaN,NaN,NaN,NaN,0.001873,NaN,NaN,NaN,NaN,0.017720,...,0.068852,NaN,NaN,NaN,NaN,0.003383,NaN,NaN,0.009195,NaN
ENSG00000000460,NaN,NaN,NaN,NaN,0.002620,NaN,NaN,0.000207,NaN,0.000105,...,0.093284,NaN,NaN,NaN,NaN,0.008790,NaN,NaN,0.003105,NaN
ENSG00000000938,NaN,NaN,NaN,NaN,0.002928,NaN,NaN,NaN,0.000762,0.012773,...,0.004300,NaN,NaN,0.000424,NaN,0.000612,NaN,NaN,0.003240,NaN
ENSG00000000971,NaN,NaN,NaN,NaN,0.002858,NaN,NaN,NaN,NaN,0.019304,...,0.005419,NaN,NaN,NaN,NaN,0.007427,NaN,NaN,0.004804,NaN


# Load OMIM silver standard

In [9]:
omim_silver_standard = pd.read_csv(os.path.join(conf.DATA_DIR, 'omim_silver_standard.tsv'), sep='\t')

In [10]:
omim_silver_standard = omim_silver_standard.dropna(subset=['ensembl_gene_id', 'trait', 'pheno_mim'])

In [11]:
display(omim_silver_standard.shape)
display(omim_silver_standard.head())

(7809, 7)

,trait,pheno_mim,mim,entry_type,entrez_gene_id,approved_gene_symbol,ensembl_gene_id
0,M41-Diagnoses_main_ICD10_M41_Scoliosis,101800,188830,gene,5573,PRKAR1A,ENSG00000108946
1,M41-Diagnoses_main_ICD10_M41_Scoliosis,102500,600275,gene,4853,NOTCH2,ENSG00000134250
2,M41-Diagnoses_main_ICD10_M41_Scoliosis,105830,601623,gene,7337,UBE3A,ENSG00000114062
3,M41-Diagnoses_main_ICD10_M41_Scoliosis,108120,190990,gene,7169,TPM2,ENSG00000198467
4,M41-Diagnoses_main_ICD10_M41_Scoliosis,108145,613629,gene,63895,PIEZO2,ENSG00000154864


# Read gwas2gene results

In [12]:
from glob import glob

import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri
pandas2ri.activate()

In [13]:
readRDS = robjects.r['readRDS']

In [14]:
f_files = glob(os.path.join(conf.OMIM_SILVER_STANDARD_GWAS_TO_GENE_DIR, '*.rds'))
display(len(f_files))

if len(f_files) != len(omim_silver_standard['trait'].unique()):
    print(f'WARNING: some files are not there. {len(omim_silver_standard["trait"].unique())} expected, {len(f_files)} found.')

99

In [15]:
gwas2genes_results = {}

for f in f_files:
    f_base = os.path.basename(f)
    f_code, _ = os.path.splitext(f_base)
    
    #print(f_base)
    rds_contents = readRDS(f)
    
    if len(rds_contents[1]) > 0:
        f_gene_list = list(rds_contents[1][0].iter_labels())
    else:
        print(f'{f_code}: empty')
        f_gene_list = []
    
    gwas2genes_results[f_code] = spredixcan_genes_associations.index.intersection(set(f_gene_list))

In [16]:
len(gwas2genes_results)

99

In [17]:
gwas2gene_all_genes = []

for k in gwas2genes_results.keys():
    gwas2gene_all_genes.extend(gwas2genes_results[k])

display(len(gwas2gene_all_genes))

gwas2gene_all_genes = set(gwas2gene_all_genes)
display(len(gwas2gene_all_genes))

# gwas2gene_all_genes = spredixcan_genes_associations.index.intersection(gwas2gene_all_genes)
# display(len(gwas2gene_all_genes))

19709

9722

# Create list of UKB-OMIM traits

In [18]:
omim_silver_standard.head()

,trait,pheno_mim,mim,entry_type,entrez_gene_id,approved_gene_symbol,ensembl_gene_id
0,M41-Diagnoses_main_ICD10_M41_Scoliosis,101800,188830,gene,5573,PRKAR1A,ENSG00000108946
1,M41-Diagnoses_main_ICD10_M41_Scoliosis,102500,600275,gene,4853,NOTCH2,ENSG00000134250
2,M41-Diagnoses_main_ICD10_M41_Scoliosis,105830,601623,gene,7337,UBE3A,ENSG00000114062
3,M41-Diagnoses_main_ICD10_M41_Scoliosis,108120,190990,gene,7169,TPM2,ENSG00000198467
4,M41-Diagnoses_main_ICD10_M41_Scoliosis,108145,613629,gene,63895,PIEZO2,ENSG00000154864


# Create PrediXcan classifier table

In [19]:
_tmp = omim_silver_standard[['trait', 'ensembl_gene_id']]
ukb_traits_common = _tmp['trait'].unique()

omim_true_classes = _tmp[['trait', 'ensembl_gene_id']].drop_duplicates()
omim_true_classes = omim_true_classes.assign(omim_value=1)
omim_true_classes = omim_true_classes.set_index(['trait', 'ensembl_gene_id'])

In [20]:
len(ukb_traits_common)

107

In [21]:
omim_true_classes.shape

(7046, 1)

In [22]:
omim_true_classes.head()

omim_value
trait                                  ensembl_gene_id            
M41-Diagnoses_main_ICD10_M41_Scoliosis ENSG00000108946           1
                                       ENSG00000134250           1
                                       ENSG00000114062           1
                                       ENSG00000198467           1
                                       ENSG00000154864           1

In [23]:
len(ukb_traits_common)

107

In [24]:
np.sum([len(k) for k in gwas2genes_results.values()])

19709

In [25]:
from entity import Trait

In [26]:
index_tuples = []

for t in ukb_traits_common:
    t_code = Trait(full_code=t).code
    if t_code not in gwas2genes_results:
        continue
    
    for g in gwas2genes_results[t_code]:
        index_tuples.append((t, g))

In [27]:
len(index_tuples)

19709

In [28]:
index_tuples[:5]

[('M41-Diagnoses_main_ICD10_M41_Scoliosis', 'ENSG00000012504'),
 ('M41-Diagnoses_main_ICD10_M41_Scoliosis', 'ENSG00000075089'),
 ('M41-Diagnoses_main_ICD10_M41_Scoliosis', 'ENSG00000075336'),
 ('M41-Diagnoses_main_ICD10_M41_Scoliosis', 'ENSG00000078401'),
 ('M41-Diagnoses_main_ICD10_M41_Scoliosis', 'ENSG00000090263')]

In [29]:
classifier_index = pd.MultiIndex.from_tuples(
    index_tuples,
    names=['ukb_efo', 'gene']
)

In [30]:
len(gwas2gene_all_genes)

9722

In [31]:
classifier_index.shape

(19709,)

In [32]:
predixcan_classifier_df = pd.DataFrame(index=classifier_index, columns=['score', 'predicted_class', 'true_class'])

In [33]:
predixcan_classifier_df = predixcan_classifier_df.sort_index()

In [34]:
predixcan_classifier_df.shape

(19709, 3)

In [35]:
predixcan_classifier_df['true_class'] = 0

In [36]:
predixcan_classifier_df.head()

score predicted_class  true_class
ukb_efo                     gene                                             
1200-Sleeplessness_insomnia ENSG00000001617   NaN             NaN           0
                            ENSG00000003756   NaN             NaN           0
                            ENSG00000004534   NaN             NaN           0
                            ENSG00000004838   NaN             NaN           0
                            ENSG00000004897   NaN             NaN           0

In [37]:
true_classes = omim_true_classes.squeeze()
display(true_classes.shape)
display(true_classes.head())

(7046,)

trait                                   ensembl_gene_id
M41-Diagnoses_main_ICD10_M41_Scoliosis  ENSG00000108946    1
                                        ENSG00000134250    1
                                        ENSG00000114062    1
                                        ENSG00000198467    1
                                        ENSG00000154864    1
Name: omim_value, dtype: int64

In [38]:
predixcan_classifier_df.loc[predixcan_classifier_df.index.intersection(true_classes.index), 'true_class'] = 1

In [39]:
assert predixcan_classifier_df['true_class'].isna().sum() == 0

In [40]:
# some testing
predixcan_classifier_df.loc[('M41-Diagnoses_main_ICD10_M41_Scoliosis',)].head()

,score,predicted_class,true_class
gene,,,
ENSG00000012504,NaN,NaN,0
ENSG00000075089,NaN,NaN,0
ENSG00000075336,NaN,NaN,0
ENSG00000078401,NaN,NaN,0
ENSG00000090263,NaN,NaN,0


In [41]:
true_classes.loc[('M41-Diagnoses_main_ICD10_M41_Scoliosis', 'ENSG00000112234')]

1

In [42]:
'ENSG00000090263' not in true_classes.loc['M41-Diagnoses_main_ICD10_M41_Scoliosis'].index

True

In [43]:
assert predixcan_classifier_df.loc[('M41-Diagnoses_main_ICD10_M41_Scoliosis', 'ENSG00000112234'), 'true_class'] == 1.0
assert predixcan_classifier_df.loc[('M41-Diagnoses_main_ICD10_M41_Scoliosis', 'ENSG00000090263'), 'true_class'] == 0.0

In [44]:
len(gwas2gene_all_genes)

9722

In [45]:
# score
df_score = pd.Series(index=classifier_index, dtype=float)

for trait in ukb_traits_common:
    trait_code = Trait(full_code=trait).code
    if trait_code not in gwas2genes_results:
        print(trait_code)
        continue
    trait_genes = gwas2genes_results[trait_code]
    if len(trait_genes) == 0:
        print(f'Empty: {trait}')
        continue
    
    scores = spredixcan_genes_associations.loc[trait_genes, trait]
    df_score.loc[trait] = scores.values

M13
I95
I71
20002_1264
20002_1081
22130
C80
20002_1538


In [46]:
# for fastENLOC fillna (because it discards results with RCP < 1e-04)
df_score = df_score.fillna(0.0).sort_index()
assert df_score.isna().sum().sum() == 0

In [47]:
df_score.head()

ukb_efo                      gene           
1200-Sleeplessness_insomnia  ENSG00000001617    0.021790
                             ENSG00000003756    0.060830
                             ENSG00000004534    0.395645
                             ENSG00000004838    0.000435
                             ENSG00000004897    0.000000
dtype: float64

In [48]:
df_score.index.get_level_values('ukb_efo').unique().shape

(99,)

In [49]:
# some testing
_gene, _trait = ('ENSG00000090263', 'M41-Diagnoses_main_ICD10_M41_Scoliosis')
assert pd.isnull(spredixcan_genes_associations.loc[_gene, _trait])
assert df_score.loc[_trait, _gene] == 0.0

In [50]:
_gene, _trait = ('ENSG00000070061', 'O14-Diagnoses_main_ICD10_O14_Gestational_pregnancyinduced_hypertension_with_significant_proteinuria')
assert pd.isnull(spredixcan_genes_associations.loc[_gene, _trait])
assert df_score.loc[_trait, _gene] == 0.0

In [51]:
df_score.dtype

dtype('float64')

In [52]:
df_score.shape

(19709,)

In [53]:
df_score.head()

ukb_efo                      gene           
1200-Sleeplessness_insomnia  ENSG00000001617    0.021790
                             ENSG00000003756    0.060830
                             ENSG00000004534    0.395645
                             ENSG00000004838    0.000435
                             ENSG00000004897    0.000000
dtype: float64

In [54]:
df_score.describe()

count    19709.000000
mean         0.023897
std          0.112420
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000213
max          1.742724
dtype: float64

In [55]:
predixcan_classifier_df = predixcan_classifier_df.assign(score=df_score)

In [56]:
# assert not predixcan_classifier_df['score'].isna().any()

In [57]:
from scipy import stats

In [58]:
_n_genes = len(gwas2gene_all_genes)
display(_n_genes)

SCORE_THRESHOLD = 0.1
display(SCORE_THRESHOLD)

9722

0.1

In [59]:
predixcan_classifier_df = predixcan_classifier_df.assign(predicted_class=(predixcan_classifier_df['score'] > SCORE_THRESHOLD).astype(int))

In [60]:
predixcan_classifier_df.shape

(19709, 3)

In [61]:
predixcan_classifier_df.head()

score  predicted_class  \
ukb_efo                     gene                                         
1200-Sleeplessness_insomnia ENSG00000001617  0.021790                0   
                            ENSG00000003756  0.060830                0   
                            ENSG00000004534  0.395645                1   
                            ENSG00000004838  0.000435                0   
                            ENSG00000004897  0.000000                0   

                                             true_class  
ukb_efo                     gene                         
1200-Sleeplessness_insomnia ENSG00000001617           0  
                            ENSG00000003756           0  
                            ENSG00000004534           0  
                            ENSG00000004838           0  
                            ENSG00000004897           0

In [62]:
predixcan_classifier_df.loc['M41-Diagnoses_main_ICD10_M41_Scoliosis'].sort_values('true_class', ascending=False).head()

,score,predicted_class,true_class
gene,,,
ENSG00000157764,0.0,0,1
ENSG00000162769,0.0,0,1
ENSG00000112234,0.0,0,1
ENSG00000012504,0.0,0,0
ENSG00000173208,0.0,0,0


## Select genes per trait

In [63]:
#selected_predixcan_classifier_df = predixcan_classifier_df.loc[predixcan_classifier_df.index.intersection(trait_genes_to_keep)]
selected_predixcan_classifier_df = predixcan_classifier_df

In [64]:
# some testing

In [65]:
selected_predixcan_classifier_df.shape

(19709, 3)

In [66]:
selected_predixcan_classifier_df.head()

score  predicted_class  \
ukb_efo                     gene                                         
1200-Sleeplessness_insomnia ENSG00000001617  0.021790                0   
                            ENSG00000003756  0.060830                0   
                            ENSG00000004534  0.395645                1   
                            ENSG00000004838  0.000435                0   
                            ENSG00000004897  0.000000                0   

                                             true_class  
ukb_efo                     gene                         
1200-Sleeplessness_insomnia ENSG00000001617           0  
                            ENSG00000003756           0  
                            ENSG00000004534           0  
                            ENSG00000004838           0  
                            ENSG00000004897           0

In [67]:
selected_predixcan_classifier_df.sort_values('predicted_class', ascending=False).head()

,,score,predicted_class,true_class
ukb_efo,gene,,,
20002_1065-Noncancer_illness_code_selfreported_hypertension,ENSG00000103510,0.786094,1,0
20002_1111-Noncancer_illness_code_selfreported_asthma,ENSG00000107485,0.859000,1,0
20002_1226-Noncancer_illness_code_selfreported_hypothyroidismmyxoedema,ENSG00000198400,0.777269,1,0
1980-Worrier_anxious_feelings,ENSG00000154319,0.169100,1,0
20002_1466-Noncancer_illness_code_selfreported_gout,ENSG00000175727,0.387800,1,0


In [68]:
_tmp = selected_predixcan_classifier_df.sort_values(['true_class', 'ukb_efo'], ascending=False)
display(_tmp.shape)
display(_tmp[_tmp['true_class'] > 0].shape)
display(_tmp[_tmp['true_class'] > 0].head())

(19709, 3)

(126, 3)

score  \
ukb_efo                                            gene                       
R69-Diagnoses_main_ICD10_R69_Unknown_and_unspec... ENSG00000068305  0.00000   
                                                   ENSG00000140443  0.00000   
R40-Diagnoses_main_ICD10_R40_Somnolence_stupor_... ENSG00000133812  0.00000   
                                                   ENSG00000183287  0.00000   
N20-Diagnoses_main_ICD10_N20_Calculus_of_kidney... ENSG00000131183  0.02435   

                                                                    predicted_class  \
ukb_efo                                            gene                               
R69-Diagnoses_main_ICD10_R69_Unknown_and_unspec... ENSG00000068305                0   
                                                   ENSG00000140443                0   
R40-Diagnoses_main_ICD10_R40_Somnolence_stupor_... ENSG00000133812                0   
                                                   ENSG00000183287                0   
N20-Diagnoses_main_ICD10_N20_Calculus_of_kidney... ENSG00000131183                0   

                                                                    true_class  
ukb_efo                                            gene                         
R69-Diagnoses_main_ICD10_R69_Unknown_and_unspec... ENSG00000068305           1  
                                                   ENSG00000140443           1  
R40-Diagnoses_main_ICD10_R40_Somnolence_stupor_... ENSG00000133812           1  
                                                   ENSG00000183287           1  
N20-Diagnoses_main_ICD10_N20_Calculus_of_kidney... ENSG00000131183           1

### Test classes

In [69]:
selected_predixcan_classifier_df.index.get_level_values('ukb_efo').unique().shape

(99,)

In [70]:
selected_predixcan_classifier_df.index.get_level_values('gene').unique().shape

(9722,)

In [71]:
_pheno = 'N20-Diagnoses_main_ICD10_N20_Calculus_of_kidney_and_ureter'

In [72]:
_clinvar_asthma_genes = omim_silver_standard[omim_silver_standard['trait'] == _pheno]['ensembl_gene_id'].unique()
display(_clinvar_asthma_genes)
display(_clinvar_asthma_genes.shape)

array(['ENSG00000075891', 'ENSG00000130600', 'ENSG00000269821',
       'ENSG00000129757', 'ENSG00000143473', 'ENSG00000275410',
       'ENSG00000019186', 'ENSG00000134371', 'ENSG00000036828',
       'ENSG00000088256', 'ENSG00000187091', 'ENSG00000122194',
       'ENSG00000138592', 'ENSG00000040531', 'ENSG00000138079',
       'ENSG00000021488', 'ENSG00000090402', 'ENSG00000131482',
       'ENSG00000137700', 'ENSG00000164007', 'ENSG00000113946',
       'ENSG00000116039', 'ENSG00000168000', 'ENSG00000123191',
       'ENSG00000171365', 'ENSG00000165704', 'ENSG00000042753',
       'ENSG00000075643', 'ENSG00000169692', 'ENSG00000109667',
       'ENSG00000131183', 'ENSG00000109062', 'ENSG00000149257',
       'ENSG00000198931', 'ENSG00000157388', 'ENSG00000167207',
       'ENSG00000124827', 'ENSG00000134873'], dtype=object)

(38,)

In [73]:
_tmp = selected_predixcan_classifier_df.loc[_pheno]
_tmp.loc[_tmp.index.intersection(_clinvar_asthma_genes)]

,score,predicted_class,true_class
ENSG00000131183,0.02435,0,1


In [74]:
_predixcan_asthma_genes = selected_predixcan_classifier_df.loc[_pheno]

In [75]:
_predixcan_asthma_genes.head()

,score,predicted_class,true_class
gene,,,
ENSG00000005187,0.0,0,0
ENSG00000005189,0.0,0,0
ENSG00000011638,0.0,0,0
ENSG00000027847,0.0,0,0
ENSG00000048140,0.0,0,0


In [76]:
selected_predixcan_classifier_df.shape

(19709, 3)

In [77]:
selected_predixcan_classifier_df['predicted_class'].value_counts()

0    18730
1      979
Name: predicted_class, dtype: int64

In [78]:
selected_predixcan_classifier_df['true_class'].value_counts()

0    19583
1      126
Name: true_class, dtype: int64

In [79]:
selected_predixcan_classifier_df.sort_values(['true_class'], ascending=[False])

score  \
ukb_efo                                            gene                        
20002_1065-Noncancer_illness_code_selfreported_... ENSG00000179218  0.000679   
                                                   ENSG00000111252  0.069440   
E66-Diagnoses_main_ICD10_E66_Obesity               ENSG00000176842  0.000000   
20002_1065-Noncancer_illness_code_selfreported_... ENSG00000111057  0.000000   
22127-Doctor_diagnosed_asthma                      ENSG00000232810  0.000000   
20002_1111-Noncancer_illness_code_selfreported_... ENSG00000124299  0.000000   
20002_1065-Noncancer_illness_code_selfreported_... ENSG00000125730  0.000602   
20002_1456-Noncancer_illness_code_selfreported_... ENSG00000196735  0.000000   
R40-Diagnoses_main_ICD10_R40_Somnolence_stupor_... ENSG00000133812  0.000000   
M05-Diagnoses_main_ICD10_M05_Seropositive_rheum... ENSG00000204498  0.000255   
E66-Diagnoses_main_ICD10_E66_Obesity               ENSG00000140718  0.001942   
20002_1381-Noncancer_illness_code_selfreported_... ENSG00000244731  0.491200   
20002_1065-Noncancer_illness_code_selfreported_... ENSG00000144061  0.000310   
20002_1111-Noncancer_illness_code_selfreported_... ENSG00000143631  0.000000   
20002_1065-Noncancer_illness_code_selfreported_... ENSG00000141646  0.000000   
I25-Diagnoses_main_ICD10_I25_Chronic_ischaemic_... ENSG00000185950  0.000000   
M06-Diagnoses_main_ICD10_M06_Other_rheumatoid_a... ENSG00000204498  0.013610   
1980-Worrier_anxious_feelings                      ENSG00000179344  0.000000   
20002_1075-Noncancer_illness_code_selfreported_... ENSG00000111252  0.072080   
20002_1065-Noncancer_illness_code_selfreported_... ENSG00000008441  0.000000   
20002_1226-Noncancer_illness_code_selfreported_... ENSG00000166037  0.000251   
20002_1065-Noncancer_illness_code_selfreported_... ENSG00000231852  0.000000   
2453-Cancer_diagnosed_by_doctor                    ENSG00000064012  0.582100   
20002_1456-Noncancer_illness_code_selfreported_... ENSG00000179344  0.079190   
20002_1381-Noncancer_illness_code_selfreported_... ENSG00000166278  0.272400   
E04-Diagnoses_main_ICD10_E04_Other_nontoxic_goitre ENSG00000178919  0.029190   
2453-Cancer_diagnosed_by_doctor                    ENSG00000003400  0.000000   
20002_1535-Noncancer_illness_code_selfreported_... ENSG00000109099  0.000000   
I25-Diagnoses_main_ICD10_I25_Chronic_ischaemic_... ENSG00000204217  0.000000   
                                                   ENSG00000130164  0.506000   
...                                                                      ...   
20002_1075-Noncancer_illness_code_selfreported_... ENSG00000108798  0.000212   
                                                   ENSG00000108511  0.000145   
                                                   ENSG00000108468  0.003465   
                                                   ENSG00000108465  0.003343   
                                                   ENSG00000108439  0.000000   
                                                   ENSG00000104892  0.000000   
                                                   ENSG00000104884  0.000000   
                                                   ENSG00000104881  0.000000   
                                                   ENSG00000104879  0.000000   
                                                   ENSG00000104866  0.000000   
                                                   ENSG00000111245  0.000000   
                                                   ENSG00000111271  0.001856   
                                                   ENSG00000120093  0.000266   
                                                   ENSG00000111275  0.000813   
                                                   ENSG00000120075  0.000301   
                                                   ENSG00000120068  0.000000   
                                                   ENSG00000118113  0.000000   
                                                   ENSG00000118007  0.0

# Save classifier table

In [80]:
# remove nans
selected_predixcan_classifier_df = selected_predixcan_classifier_df.dropna()

In [81]:
selected_predixcan_classifier_df.head()

score  predicted_class  \
ukb_efo                     gene                                         
1200-Sleeplessness_insomnia ENSG00000001617  0.021790                0   
                            ENSG00000003756  0.060830                0   
                            ENSG00000004534  0.395645                1   
                            ENSG00000004838  0.000435                0   
                            ENSG00000004897  0.000000                0   

                                             true_class  
ukb_efo                     gene                         
1200-Sleeplessness_insomnia ENSG00000001617           0  
                            ENSG00000003756           0  
                            ENSG00000004534           0  
                            ENSG00000004838           0  
                            ENSG00000004897           0

In [82]:
selected_predixcan_classifier_df.shape

(19709, 3)

In [83]:
selected_predixcan_classifier_df.to_csv(
    os.path.join(output_dir, 'fastenloc-torus-classifier_data.tsv.gz'),
    sep='\t', index=False
)